In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

c:\Users\heegyukim\Desktop\project\open-domain-dialog


In [2]:
from odd.gpt import utils
from transformers import AutoTokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
tokenizer.add_special_tokens({
    "bos_token": "<s>",
    "eos_token": "</s>",
    "pad_token": "<pad>",
    "sep_token": "\n"
})
tokenizer

PreTrainedTokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<|endoftext|>', 'sep_token': '\n', 'pad_token': '<pad>'})

In [16]:
# tokenizer.encode("\n")
tokenizer.encode("안녕하세요\n")
# tokenizer.tokenize("안녕하세요 \n ㅋ")

[25906, 8702, 7801, 8084, 375]

In [11]:
texts = [
    "안녕하세요",
    "반가 반가워요 반갑스비다 크크루삥ㅂ뽕zzzㅋㅋㅋㅋㅋzz"
]

out = utils.prepare_batch(
    tokenizer,
    texts,
    16,
    "cpu",
    # add_bos_token=True,
    add_eos_token=True
)
print(out)


print(tokenizer.batch_decode(out["input_ids"]))
print(tokenizer.batch_decode(out["labels"].masked_fill(out["labels"] == -100, 3)))
print(out["attention_mask"].shape)


{'input_ids': tensor([[25906,  8702,  7801,  8084,     1,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3],
        [36230, 36230,  8102,  8084,  9138,  6831,  7877,  7692,  7182,  9297,
          8505,  7445,  7755,   597,  7739,   464]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ 8702,  7801,  8084,     1,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [36230,  8102,  8084,  9138,  6831,  7877,  7692,  7182,  9297,  8505,
          7445,  7755,   597,  7739,   464,   464]])}
['안녕하세요</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>', '반가 반가워요 반갑스비다 크크루삥ᄇ뽕z']
['하세요</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>', '반가워요 반갑스비다 크크루삥ᄇ뽕zz']
torch.Size([2, 16])


In [4]:
from odd.gpt.gpt import GPTTask
from hydra.core.global_hydra import GlobalHydra
from hydra import compose, initialize

if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance()
initialize("../config")
config = compose("gpt")
config

{'project': 'open-domain-dialog', 'run_name': 'gpt2-dialog', 'logger': {'name': 'wandb', 'val_sample_batches': 10, 'val_sample_generate_params': {'num_beams': 4, 'repetition_penalty': 2.0, 'max_new_tokens': 32}}, 'dataset': {'train': {'shuffle': True, 'use_auth_token': True, 'paths': ['heegyu/nikl_online_conv_2022']}, 'validation': {'use_auth_token': True, 'split': 'test', 'paths': ['heegyu/aihub_daily_conv_2022_CRF']}}, 'optimizer': {'cls': 'adamw', 'learning_rate': 0.001}, 'trainer': {'train_epochs': 3, 'train_batch_size': 16, 'accumulate_grad_batches': 16, 'limit_val_batches': 200, 'eval_batch_size': 4, 'num_sanity_val_steps': 1, 'check_val_every_n_epoch': 1}, 'model': {'plm': 'skt/kogpt2-base-v2', 'max_seq_len': 512}, 'tokenizer': {'add_special_tokens': {'pad_token': '<pad>', 'bos_token': '<s>', 'eos_token': '</s>'}}}

In [5]:
gpt = GPTTask(config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
gpt.generate(
    ["안녕하세요??", "반가워요, 오늘은 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ"],
    # ["안녕하"],
    # max_new_tokens=32,
    num_beams=4,
    repetition_penalty=2.0
)

Input length of input_ids is 75, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


RuntimeError: The expanded size of the tensor (20) must match the existing size (76) at non-singleton dimension 0.  Target sizes: [20].  Tensor sizes: [76]

In [19]:
gpt.step(["안녕핫ㅇ", "방방가가가워요"])

torch.Size([2, 8, 51200]) torch.Size([2, 8])
logits
targets
loss


tensor(10.7274, grad_fn=<NllLossBackward0>)

In [7]:
gpt.training_step(
    {
        "utterances": [["안녕", "하세요"], ["반갑", "습", "니닼ㅋ"]]
    }, 0
)

KeyError: 'dialog'

In [50]:
gpt.validation_step(
    {
        "context": ["아 오늘 뭐함", "그러게"],
        "response": ["아 오늘 뭐함", "그러게"]
    }, 0
)

['아 오늘 뭐함</s>아 오늘 뭐함</s>', '그러게</s>그러게</s>']
['아 오늘 뭐함</s>', '그러게</s>']


{'context': ['아 오늘 뭐함</s>', '그러게</s>'],
 'response': ['아 오늘 뭐함</s>', '그러게</s>'],
 'prediction': ['<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>',
  '<pad><pad> <unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>']}

In [53]:
gpt._join_uttrs([["안ㄴ", "ㅋㅋ"], ["??", "ㅎㅎㅎ"]], [[1,2,],[1,2,]])

['1 : 안ㄴ</s>2 : ㅋㅋ</s>', '1 : ??</s>2 : ㅎㅎㅎ</s>']